In [ ]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_data.head()

# 3/13  2. Базовые операции со столбцами DataFrame 

СОЗДАНИЕ КОПИИ ТАБЛИЦЫ

In [ ]:
melb_df = melb_data.copy()
melb_df.head()

УДАЛЕНИЕ СТОЛБЦОВ

За удаление строк и столбцов в таблице отвечает метод `drop()`.  

Удалим столбцы `index` и `Coordinates` из таблицы с помощью метода `drop()`. Выведем первые пять строк таблицы и убедимся, что всё прошло успешно.

Основные параметры метода `.drop()`

`labels` — порядковые номера или имена столбцов, которые подлежат удалению; если их несколько, то передаётся список;  
`axis` — ось совершения операции, `axis=0` — удаляются строки, `axis=1` — удаляются столбцы;  
`inplace` — если параметр выставлен на `True`, происходит замена изначального DataFrame на новый, при этом метод ничего не возвращает; если на `False` — возвращается копия DataFrame, из которой удалены указанные строки (столбцы), при этом первоначальный DataFrame не изменяется; по умолчанию параметр равен `False`.

In [ ]:
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)# создание копии с изменениями
melb_df.head()

# Альтернативный вариант:

# melb_df.drop(['index','Coordinates'],axis=1,inplace=True)# меняет объект
# melb_df.head()

МАТЕМАТИЧЕСКИЕ ОПЕРАЦИИ СО СТОЛБЦАМИ

→ `Pandas` поддерживает базовые математические операции между столбцами: **столбцы можно складывать, вычитать, умножать, делить между собой, а также возводить в степень**. С помощью таких операций мы можем создавать новые признаки или производить преобразования над старыми.

Например, давайте создадим переменную `total_rooms`, в которой будем хранить общее количество комнат в здании. Для этого выполним сложение столбцов с количеством комнат, ванн и спален:

In [ ]:
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
display(total_rooms)

А теперь введём признак `MeanRoomsSquare`, который соответствует средней площади одной комнаты для каждого объекта. Для этого разделим площадь здания на полученное ранее общее количество комнат:

In [ ]:
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
display(melb_df['MeanRoomsSquare'])

Можно ввести ещё один интересный признак — `AreaRatio`, коэффициент соотношения площади здания (`BuildingArea`) и площади участка (`Landsize`). Для этого разницу двух площадей поделим на их сумму:

In [ ]:
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'])

Что показывает такой коэффициент? Если присмотреться, можно увидеть, что `AreaRatio` лежит в интервале от -1 до 1.

Рассмотрим три случая, чтобы понять его значение:

Если рассматриваемые площади равны, то числитель дроби зануляется и коэффициент тоже равен 0.  
Если площадь участка больше площади здания, то коэффициент начинает расти в отрицательную сторону, и в положительную сторону, если наоборот.  
Наконец, в предельном случае, если площадь здания равна 0, то числитель дроби равен знаменателю со знаком минус, а коэффициент равен -1. Если площадь участка равна 0, то числитель дроби равен знаменателю со знаком плюс, а коэффициент равен 1.  
Таким образом, значение в столбце `AreaRatio` служит своеобразным указателем соотношения площадей объекта недвижимости. Для пустырей — участков без строений — он будет равен -1, для домов без территории — 1, во всех остальных случаях мы можем увидеть, какая площадь больше — здания или участка.

# 4/13  3. Работа с датами в DataFrame

ФОРМАТ DATETIME

Единым способом обозначения даты и времени в Pandas является формат `datetime`, который записывается как `YYYY-MM-DD HH: MM: SS`, то есть составляющие времени указываются в следующем порядке: `год, месяц, день, час, минута, секунда`. 

В наших данных дата записана в виде DD/MM/YYYY, например 3/12/2017. Посмотрим на это:

In [ ]:
display(melb_df['Date'])

Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах, в формат `datetime`, можно воспользоваться функцией `pandas.to_datetime()`. В нашем случае в функции нужно указать параметр `dayfirst=True`, который будет обозначать, что в первоначальном признаке первым идёт день. Преобразуем столбец `Date` в формат `datetime`, передав его в эту функцию:

`pandas.to_datetime`(`arg`, `errors`='raise', `dayfirst`=False, `yearfirst`=False, `utc`=False, `format`=None, `exact`=_NoDefault.no_default, `unit`=None, `infer_datetime_format`=_NoDefault.no_default, `origin`='unix', `cache`=True)

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

ВЫДЕЛЕНИЕ АТРИБУТОВ DATETIME

Тип данных `datetime` позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

`date` — дата;  
`year`, `month`, `day` — год, месяц, день;  
`time` — время;  
`hour`, `minute`, `second` — час, минута, секунда;  
`dayofweek` — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;  
`day_name` — название дня недели;  
`dayofyear` — порядковый день года;  
`quarter` — квартал (интервал в три месяца).  

Аксессор — это атрибут столбца, хранящий переменные, которые были строковым представлением времени, а затем были изменены с помощью `pd.to_datetime()`.

Обратите внимание, что вы не сможете обратиться к аксессору, если ваш столбец не приведён к типу `datetime`.

Например, обратившись по атрибуту `dt.year` в столбце `Date`, мы можем «достать» год продажи и понять, за какой интервал времени (в годах) представлены наши данные, а также на какой год приходится наибольшее число продаж:

In [ ]:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. Для этого выделим атрибут `dt.month` и на этот раз занесём результат в столбец `MonthSale`, а затем найдём относительную частоту продаж для каждого месяца от общего количества продаж — для этого используем метод `value_counts()` с параметром `normalize` (вывод в долях):

In [ ]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

РАБОТА С ИНТЕРВАЛАМИ

In [ ]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)

В результате мы получаем `Series`, элементом которого является количество дней, которое прошло с 1 января 2016 года. Обратите внимание, что данные такого формата относятся к типу `timedelta`.

Чтобы превратить количество дней из формата интервала в формат целого числа дней, можно воспользоваться аксессором `dt` для формата `timedelta` и извлечь из него атрибут `days`:

In [ ]:
display(delta_days.dt.days)


Рассмотрим другой пример. Давайте создадим признак возраста объекта недвижимости в годах на момент продажи. Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. Результат оформим в виде столбца `AgeBuilding`:

In [ ]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])

**Примечание**. Обратите внимание, что, так как года кодируются целым числом, в результате мы тоже получаем целочисленный столбец — тип int64 (а не timedelta).

На самом деле столбец `AgeBuilding` дублирует информацию столбца `YearBuilt`, так как, зная год постройки здания, мы автоматически знаем его возраст. Такие признаки не стоит оставлять вместе, поэтому оставим возраст здания, так как он является более наглядным, а год постройки удалим из таблицы:

In [ ]:
melb_df = melb_df.drop('YearBuilt', axis=1)
melb_df

Задание 3.3

Создайте в таблице `melb_df` признак `WeekdaySale` (день недели). Найдите, сколько объектов недвижимости было продано в выходные (суббота и воскресенье), результат занесите в переменную `weekend_count`. В качестве ответа введите результат вывода переменной `weekend_count`.


In [ ]:
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df['WeekdaySale'].value_counts()[5] + melb_df['WeekdaySale'].value_counts()[6]
weekend_count

Метод применяется для отображения разницы `.diff()`.  
pandas.DataFrame.diff  
DataFrame.diff(periods=1, axis=0)

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'a': [1, 2, 3, 4, 5, 6],
                   'b': [1, 1, 2, 3, 5, 8],
                   'c': [1, 4, 9, 16, 25, 36]})
df
df.diff() # Difference with previous row
# df.diff(axis=1) # Difference with previous column
# df.diff(periods=3) # Difference with 3rd previous row
# df.diff(periods=-1) # Difference with following row

# # Overflow in input dtype
# df = pd.DataFrame({'a': [1, 0]}, dtype=np.uint8)
# df.diff()

# 5/13  4. Создание и преобразование столбцов с помощью функций

Мы можем написать некоторую функцию, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат. Далее мы можем применить эту функцию к каждому элементу в столбце с помощью специального метода `apply()`. В результате применения этой функции будет возвращён объект `Series`, элементы которого будут представлять результат работы этой функции.



Вычислим количество уникальных значений в столбце с помощью метода `nunique()`:

In [ ]:
import pandas as pd

print(melb_df['Address'].nunique())
# 13378

In [ ]:
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])
# 2/119 Railway St N
# 9/400 Dandenong Rd
# 172 Danks St

Итак, адрес строится следующим образом: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна. Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:

In [ ]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию `get_street_type` в аргумент метода столбца `apply()`. В результате получим объект `Series`, который положим в переменную `street_types`:

In [ ]:
street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

Уберем повторяющиеся названия

In [ ]:
# На вход данной функции поступает строка с адресом.
def get_street_name(address):

    if address[:2] == 'Av':
        address = 'Av'
    if address == 'Bvd' or address == 'Boulevard':
        address = 'Bvd'
    if address == 'Pde' or address == 'Parade':
        address = 'Pde'
    return address

In [ ]:
street_types = street_types.apply(get_street_name)
display(street_types)

Обратите внимание, что функция пишется для одного элемента столбца, а метод `apply()` применяется к каждому его элементу. Используемая функция обязательно должна иметь возвращаемое значение.

Итак, мы смогли выделить подтип улицы. Посмотрим, сколько уникальных значений у нас получилось:

In [ ]:
print(street_types.nunique())
# 56

У нас есть 56 уникальных значений. Однако наш результат можно улучшить. Давайте для начала посмотрим на частоту каждого подтипа улицы с помощью метода `value_counts`:

In [ ]:
display(street_types.value_counts())

Из данного вывода можно увидеть, что есть группа наиболее популярных подтипов улиц, а дальше частота подтипов быстро падает.

В таком случае давайте применим очень распространённый метод уменьшения количества уникальных категорий — выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как '`other`' (другие).

Для этого к результату метода `value_counts` применим метод `nlargest()`, который возвращает `n` наибольших значений из `Series`. Зададим `n=10`, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута `index`, а результат занесём в переменную `popular_stypes`:

In [ ]:
popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)
# Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём `lambda`-функцию, которая будет проверять, есть ли строка `x` в этом перечне, и, если это так, `lambda`-функция будет возвращать `x`, в противном случае она будет возвращать строку '`other`'. Наконец, применим такую функцию к `Series` `street_types`, полученному ранее, а результат определим в новый столбец таблицы `StreetType`:

In [ ]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])

Посмотрим на результирующее число уникальных подтипов:

In [ ]:
print(melb_df['StreetType'].nunique())
# 11

Теперь у нас нет потребности хранить признак `Address`, так как если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами `Longitude` и `Lattitude`. Удалим его из нашей таблицы:

In [ ]:
melb_df = melb_df.drop('Address', axis=1)

Резюмируя, поделимся общими рекомендациями по уменьшению числа уникальных значений в признаке, который описывается категориями:

1  
Определите (хотя бы на глаз) соотношение числа уникальных категорий интересующего вас признака к общему числу объектов в таблице. Если это соотношение превышает значение 30 %, то это уже повод задуматься над уменьшением числа категорий и перейти к шагу 2.

2  
Если ваш признак уникален для каждого объекта, например адрес, имя или название, то такой признак, скорее всего, не имеет статистической значимости. От таких признаков чаще всего избавляются. Однако можно попробовать выделить из этого признака какие-то общие черты, например, как мы это сделали с подтипами улиц. То же самое можно сделать, например, с названиями компаний, в которых может быть скрыт признак типа организации (из строки «ООО Три Слепые Мыши» можно извлечь ООО — общество с ограниченной ответственностью).

Далее переходите к шагу 3.

3  
Если даже после преобразования число уникальных категорий всё ещё велико, можно попробовать с помощью метода value_counts() оценить, есть ли в данных категории, которые употребляются гораздо реже, чем остальные. Если такие категории присутствуют, переходите к шагу 4.

4  
Можно подобрать число  популярных категорий таким образом, чтобы эти категории покрывали большую часть ваших данных.

Когда вы выбрали оптимальное число, переходите к шагу 5.

5  
Наконец, можно совершить преобразование, обозначив категории, не попавшие в число популярных, как «другие».

**→ Такая методика является очень популярной и хорошо показывает себя на практике. Однако не нужно ей злоупотреблять: применяя эту методику ко всем столбцам подряд, вы рискуете потерять немалую долю полезной информации из ваших данных. Внимательно изучите интересующий вас признак, прежде чем преобразовывать его.**